Try Apache Beam - Java
In this notebook, we set up a Java development environment and work through a simple example using the DirectRunner. You can explore other runners with the Beam Capatibility Matrix.

To navigate through different sections, use the table of contents. From View drop-down list, select Table of contents.

To run a code cell, you can click the Run cell button at the top left of the cell, or by select it and press Shift+Enter. Try modifying a code cell and re-running it to see what happens.

To learn more about Colab, see Welcome to Colaboratory!.



In [1]:
# https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/get-started/try-apache-beam-java.ipynb#scrollTo=CgTXBdTsBn1F
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}  # This is magic to run 'cmd' in the shell.
  print('')


In [2]:
# Copy the input file into the local filesystem.
run('mkdir -p data')
run('gsutil cp gs://dataflow-samples/shakespeare/kinglear.txt data/')
run('mkdir -p src/main/java/samples/quickstart')
print('Done')

>> mkdir -p data

>> gsutil cp gs://dataflow-samples/shakespeare/kinglear.txt data/
Copying gs://dataflow-samples/shakespeare/kinglear.txt...
/ [1 files][153.6 KiB/153.6 KiB]                                                
Operation completed over 1 objects/153.6 KiB.                                    

>> mkdir -p src/main/java/samples/quickstart

Done


In [3]:
# Update and upgrade the system before installing anything else.
# run('apt-get update > /dev/null')
# run('apt-get upgrade > /dev/null')

# # Install the Java JDK.
# run('apt-get install -y default-jdk > /dev/null')

run('apt-get update')
run('apt-get upgrade -y')

# Install the Java JDK.
run('apt-get install -y default-jdk')

# Check the Java version to see if everything is working well.
run('javac -version')
print('Done')

>> apt-get update
Get:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5386 B]
Get:2 https://packages.cloud.google.com/apt cloud-sdk InRelease [6739 B]       
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:5 http://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [816 B]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Get:7 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [203 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1335 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [733 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [828 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages

In [4]:
import os

# Download the gradle source.
gradle_version = 'gradle-5.0'
gradle_path = f"/opt/{gradle_version}"
if not os.path.exists(gradle_path):
  run(f"wget -q -nc -O gradle.zip https://services.gradle.org/distributions/{gradle_version}-bin.zip")
  run('unzip -q -d /opt gradle.zip')
  run('rm -f gradle.zip')

# We're choosing to use the absolute path instead of adding it to the $PATH environment variable.
def gradle(args):
  run(f"{gradle_path}/bin/gradle --console=plain {args}")

gradle('-v')
print('Done')

>> wget -q -nc -O gradle.zip https://services.gradle.org/distributions/gradle-5.0-bin.zip

>> unzip -q -d /opt gradle.zip

>> rm -f gradle.zip

>> /opt/gradle-5.0/bin/gradle --console=plain -v

Welcome to Gradle 5.0!

Here are the highlights of this release:
 - Kotlin DSL 1.0
 - Task timeouts
 - Dependency alignment aka BOM support
 - Interactive `gradle init`

For more details see https://docs.gradle.org/5.0/release-notes.html


------------------------------------------------------------
Gradle 5.0
------------------------------------------------------------

Build time:   2018-11-26 11:48:43 UTC
Revision:     7fc6e5abf2fc5fe0824aec8a0f5462664dbcd987

Kotlin DSL:   1.0.4
Kotlin:       1.3.10
Groovy:       2.5.4
Ant:          Apache Ant(TM) version 1.9.13 compiled on July 10 2018
JVM:          1.8.0_92 (Azul Systems, Inc. 25.92-b15)
OS:           Linux 4.19.0-18-cloud-amd64 amd64


Done


In [5]:
%%writefile build.gradle

plugins {
  // id 'idea'     // Uncomment for IntelliJ IDE
  // id 'eclipse'  // Uncomment for Eclipse IDE

  // Apply java plugin and make it a runnable application.
  id 'java'
  id 'application'

  // 'shadow' allows us to embed all the dependencies into a fat jar.
  id 'com.github.johnrengelman.shadow' version '4.0.3'
}

// This is the path of the main class, stored within ./src/main/java/
mainClassName = 'samples.quickstart.WordCount'

// Declare the sources from which to fetch dependencies.
repositories {
  mavenCentral()
}

// Java version compatibility.
sourceCompatibility = 1.8
targetCompatibility = 1.8

// Use the latest Apache Beam major version 2.
// You can also lock into a minor version like '2.9.+'.
ext.apacheBeamVersion = '2.+'

// Declare the dependencies of the project.
dependencies {
  shadow "org.apache.beam:beam-sdks-java-core:$apacheBeamVersion"

  runtime "org.apache.beam:beam-runners-direct-java:$apacheBeamVersion"
  runtime "org.slf4j:slf4j-api:1.+"
  runtime "org.slf4j:slf4j-jdk14:1.+"

  testCompile "junit:junit:4.+"
}

// Configure 'shadowJar' instead of 'jar' to set up the fat jar.
shadowJar {
  baseName = 'WordCount' // Name of the fat jar file.
  classifier = null       // Set to null, otherwise 'shadow' appends a '-all' to the jar file name.
  manifest {
    attributes('Main-Class': mainClassName)  // Specify where the main class resides.
  }
}


Writing build.gradle


In [6]:
%%writefile src/main/java/samples/quickstart/WordCount.java

package samples.quickstart;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.io.TextIO;
import org.apache.beam.sdk.options.PipelineOptions;
import org.apache.beam.sdk.options.PipelineOptionsFactory;
import org.apache.beam.sdk.transforms.Count;
import org.apache.beam.sdk.transforms.Filter;
import org.apache.beam.sdk.transforms.FlatMapElements;
import org.apache.beam.sdk.transforms.MapElements;
import org.apache.beam.sdk.values.KV;
import org.apache.beam.sdk.values.TypeDescriptors;

import java.util.Arrays;

public class WordCount {
  public static void main(String[] args) {
    String inputsDir = "data/*";
    String outputsPrefix = "outputs/part";

    PipelineOptions options = PipelineOptionsFactory.fromArgs(args).create();
    Pipeline pipeline = Pipeline.create(options);
    pipeline
        .apply("Read lines", TextIO.read().from(inputsDir))
        .apply("Find words", FlatMapElements.into(TypeDescriptors.strings())
            .via((String line) -> Arrays.asList(line.split("[^\\p{L}]+"))))
        .apply("Filter empty words", Filter.by((String word) -> !word.isEmpty()))
        .apply("Count words", Count.perElement())
        .apply("Write results", MapElements.into(TypeDescriptors.strings())
            .via((KV<String, Long> wordCount) ->
                  wordCount.getKey() + ": " + wordCount.getValue()))
        .apply(TextIO.write().to(outputsPrefix));
    pipeline.run();
  }
}

Writing src/main/java/samples/quickstart/WordCount.java


In [7]:
# Build the project.
gradle('build')

# Check the generated build files.
run('ls -lh build/libs/')
print('Done')

>> /opt/gradle-5.0/bin/gradle --console=plain build
Starting a Gradle Daemon (subsequent builds will be faster)
> Task :compileJava
> Task :processResources NO-SOURCE
> Task :classes
> Task :jar
> Task :startScripts
> Task :distTar
> Task :distZip
> Task :shadowJar
> Task :startShadowScripts
> Task :shadowDistTar
> Task :shadowDistZip
> Task :assemble
> Task :compileTestJava NO-SOURCE
> Task :processTestResources NO-SOURCE
> Task :testClasses UP-TO-DATE
> Task :test NO-SOURCE
> Task :check UP-TO-DATE
> Task :build

BUILD SUCCESSFUL in 42s
9 actionable tasks: 9 executed

>> ls -lh build/libs/
total 43M
-rw-r--r-- 1 root root 3.0K Dec 23 06:30 Dataflowclass1.jar
-rw-r--r-- 1 root root  43M Dec 23 06:30 WordCount.jar

Done


In [8]:
# Run the shadow (fat jar) build.
gradle('runShadow')

# Sample the first 20 results, remember there are no ordering guarantees.
run('head -n 20 outputs/part-00000-of-*')
print('Done')

>> /opt/gradle-5.0/bin/gradle --console=plain runShadow
> Task :compileJava UP-TO-DATE
> Task :processResources NO-SOURCE
> Task :classes UP-TO-DATE
> Task :shadowJar UP-TO-DATE
> Task :startShadowScripts UP-TO-DATE
> Task :installShadowDist

> Task :runShadow
Dec 23, 2021 6:30:45 AM org.apache.beam.sdk.io.FileBasedSource getEstimatedSizeBytes
INFO: Filepattern data/* matched 1 files with total size 157283
Dec 23, 2021 6:30:45 AM org.apache.beam.sdk.io.FileBasedSource split
INFO: Splitting filepattern data/* into bundles of size 39320 took 1 ms and produced 1 files and 4 bundles
Dec 23, 2021 6:31:56 AM org.apache.beam.sdk.io.WriteFiles$WriteShardsIntoTempFilesFn processElement
INFO: Opening writer f2b8de4a-35d1-40ca-944f-fb4f08c71411 for window org.apache.beam.sdk.transforms.windowing.GlobalWindow@7098b907 pane PaneInfo{isFirst=true, isLast=true, timing=ON_TIME, index=0, onTimeIndex=0} destination null
Dec 23, 2021 6:31:56 AM org.apache.beam.sdk.io.WriteFiles$WriteShardsIntoTempFilesFn

In [ ]:
# You can now distribute and run your Java application as a standalone jar file.
run('cp build/libs/WordCount.jar .')
run('java -jar WordCount.jar')

# Sample the first 20 results, remember there are no ordering guarantees.
run('head -n 20 outputs/part-00000-of-*')
print('Done')

In [ ]:
! rm outputs/*
java_run()
run('head -n 20 outputs/part-00000-of-*')
print('Done')

In [ ]:
# Build the project.
gradle('build')

# Check the generated build files.
run('ls -lh build/libs/')

# # Run the shadow (fat jar) build.
gradle('runShadow')

# # Sample the first 20 results, remember there are no ordering guarantees.
run('head -n 20 outputs/regions*')

print('Done')
